In [1]:
from bayes_opt import BayesianOptimization
from bayes_opt import UtilityFunction

In [2]:
def dummy_func(x, y):
    
    return pow(x, 2) - pow(y - 1, 2) + 1

In [3]:
optimizer = BayesianOptimization(
                    f       = None,
                    pbounds = {'x' : (-2, 2), 'y' : (-3, 3)},
                    verbose = 2, random_state = 1
                )

utility    = UtilityFunction(kind = 'ucb', kappa = 2.5, xi = 0.0)
next_point = optimizer.suggest(utility)

print(f'next point to probe is       : {next_point}')
target = dummy_func(**next_point)

print(f'found the target value to be : {target}')

next point to probe is       : {'x': -0.331911981189704, 'y': 1.3219469606529488}
found the target value to be : 1.006515717783603


In [4]:
optimizer.register(params = next_point, target= target)

for _ in range(5):
    
    next_point = optimizer.suggest(utility)
    target     = dummy_func(**next_point)
    optimizer.register(params = next_point, target = target)
    
    print(f'target     : {target:.2f}')
    print(f'next point : {next_point}\n')
    
print(optimizer.max)

target     : -11.38
next point : {'x': 1.8861546000771092, 'y': -2.9917780942581977}

target     : 0.99
next point : {'x': -0.31764604716962586, 'y': 1.3285597809731806}

target     : 1.00
next point : {'x': -2.0, 'y': 3.0}

target     : -0.18
next point : {'x': -2.0, 'y': -1.2761469781507055}

target     : 1.00
next point : {'x': 2.0, 'y': 3.0}

{'target': 1.006515717783603, 'params': {'x': -0.331911981189704, 'y': 1.3219469606529488}}


In [5]:
def func_with_disc_params(x, y, d):
    
    assert type(d) == int
    return ((x + y + d) // (1 + d)) / (1 +(x + y)**2)


def function_optimize(x, y, w):
    
    return func_with_disc_params(x, y, int(w))

In [6]:
optimizer = BayesianOptimization(
                    f       = function_optimize,
                    pbounds = {'x' : (-10, 10), 'y' : (-10, 10), 'w' :(0, 5)},
                    verbose = 2, random_state = 1
                )

optimizer.set_gp_params(alpha = 1e-3)
optimizer.maximize()

|   iter    |  target   |     w     |     x     |     y     |
-------------------------------------------------------------
| 1         | -0.06199  | 2.085     | 4.406     | -9.998    |
| 2         | -0.0344   | 1.512     | -7.065    | -8.153    |
| 3         | -0.2177   | 0.9313    | -3.089    | -2.065    |
| 4         | 0.1865    | 2.694     | -1.616    | 3.704     |
| 5         | -0.2187   | 1.022     | 7.562     | -9.452    |
| 6         | 0.1868    | 2.533     | -1.728    | 3.815     |
| 7         | 0.05119   | 3.957     | -0.6151   | 6.785     |
| 8         | 0.1761    | 0.5799    | 1.181     | 4.054     |
| 9         | 0.04045   | 4.004     | 4.304     | 2.656     |
| 10        | 0.07509   | 0.0       | 4.843     | 7.759     |
| 11        | 0.3512    | 0.0       | -5.713    | 7.072     |
| 12        | -0.8068   | 0.0       | -9.09     | 8.6       |
| 13        | 0.3774    | 0.3974    | -4.19     | 6.264     |
| 14        | 0.157     | 0.0       | -3.587    | 8.534     |
| 15    

In [7]:
import os

import pandas as pd
import numpy as np

In [8]:
ROOT_PATH = '/'.join(os.getcwd().split(os.path.sep)[:-2])
DATA_PATH = f'{ROOT_PATH}/dataset/psychology'

In [9]:
train_df   = pd.read_csv(f'{DATA_PATH}/train.csv', index_col = 0)
test_df    = pd.read_csv(f'{DATA_PATH}/test_x.csv', index_col = 0)
submission = pd.read_csv(f'{DATA_PATH}/sample_submission.csv', index_col = 0) 

In [10]:
feats      = train_df.drop('voted', axis = 1)
target     = train_df['voted']

In [11]:
feats.race.value_counts()

White                    31248
Asian                     6834
Other                     4330
Black                     2168
Native American            548
Arab                       351
Indigenous Australian       53
Name: race, dtype: int64

In [12]:
feats.age_group.value_counts()

10s     14215
20s     14112
30s      7836
40s      5051
50s      2889
60s      1194
+70s      235
Name: age_group, dtype: int64

In [13]:
feats.gender.value_counts()

Male      24217
Female    21315
Name: gender, dtype: int64

In [14]:
feats.religion.value_counts()

Atheist                 10192
Agnostic                 9624
Christian_Catholic       6431
Christian_Other          5137
Christian_Protestant     4875
Other                    4770
Hindu                    1429
Muslim                   1192
Buddhist                  850
Jewish                    487
Christian_Mormon          428
Sikh                      117
Name: religion, dtype: int64

In [15]:
print(f'원본 데이터 칼럼 : {list(feats.columns)}\n')
feats_dummies = pd.get_dummies(feats)
print(f'더미화 된 데이터 칼럼 : {list(feats_dummies.columns)}')

원본 데이터 칼럼 : ['QaA', 'QaE', 'QbA', 'QbE', 'QcA', 'QcE', 'QdA', 'QdE', 'QeA', 'QeE', 'QfA', 'QfE', 'QgA', 'QgE', 'QhA', 'QhE', 'QiA', 'QiE', 'QjA', 'QjE', 'QkA', 'QkE', 'QlA', 'QlE', 'QmA', 'QmE', 'QnA', 'QnE', 'QoA', 'QoE', 'QpA', 'QpE', 'QqA', 'QqE', 'QrA', 'QrE', 'QsA', 'QsE', 'QtA', 'QtE', 'age_group', 'education', 'engnat', 'familysize', 'gender', 'hand', 'married', 'race', 'religion', 'tp01', 'tp02', 'tp03', 'tp04', 'tp05', 'tp06', 'tp07', 'tp08', 'tp09', 'tp10', 'urban', 'wf_01', 'wf_02', 'wf_03', 'wr_01', 'wr_02', 'wr_03', 'wr_04', 'wr_05', 'wr_06', 'wr_07', 'wr_08', 'wr_09', 'wr_10', 'wr_11', 'wr_12', 'wr_13']

더미화 된 데이터 칼럼 : ['QaA', 'QaE', 'QbA', 'QbE', 'QcA', 'QcE', 'QdA', 'QdE', 'QeA', 'QeE', 'QfA', 'QfE', 'QgA', 'QgE', 'QhA', 'QhE', 'QiA', 'QiE', 'QjA', 'QjE', 'QkA', 'QkE', 'QlA', 'QlE', 'QmA', 'QmE', 'QnA', 'QnE', 'QoA', 'QoE', 'QpA', 'QpE', 'QqA', 'QqE', 'QrA', 'QrE', 'QsA', 'QsE', 'QtA', 'QtE', 'education', 'engnat', 'familysize', 'hand', 'married', 'tp01', 'tp02', 'tp03'

In [16]:
print(f'X shape : {feats.shape}, dummies shape : {feats_dummies.shape}')

X shape : (45532, 76), dummies shape : (45532, 100)


In [17]:
test_df = pd.get_dummies(test_df)
test_df.shape

(11383, 100)

In [18]:
feats_dummies = feats_dummies.fillna(feats_dummies.mean())
feats_dummies.drop_duplicates(keep = 'first', inplace = True)
feats_dummies.shape

(45532, 100)

In [19]:
from sklearn.metrics import roc_auc_score,make_scorer
from sklearn.model_selection import cross_validate
from sklearn.preprocessing import MinMaxScaler
from bayes_opt import BayesianOptimization
import lightgbm as lgbm

In [20]:
scaler = MinMaxScaler()
scaler.fit(feats_dummies)

feats = scaler.transform(feats_dummies)
test  = scaler.transform(test_df)

In [21]:
def lgbm_cv(lr, num_leaves, max_depth, min_child_weight, colsample_bytree,
            feature_fraction, bagging_fraction, lambda_l1, lambda_l2):
    
    model = lgbm.LGBMClassifier(learning_rate = lr,
                                n_estimators  = 300,
                                num_leaves    = int(round(num_leaves)),
                                max_depth     = int(round(max_depth)),
                                min_child_weight = int(round(min_child_weight)),
                                colsample_bytree = colsample_bytree,
                                feature_fraction = max(min(feature_fraction, 1), 0),
                                bagging_fraction = max(min(bagging_fraction, 1), 0),
                                lambda_l1        = max(lambda_l1, 0),
                                lambda_l2        = max(lambda_l2, 0)
                            )
    
    scoring   = {'roc_auc_score' : make_scorer(roc_auc_score)}
    result    = cross_validate(model, feats, target, cv = 5, scoring = scoring)
    auc_score = result['test_roc_auc_score'].mean()
    
    return auc_score

In [22]:
pbounds = {
            'lr'            : (0.0001, 0.05),
            'num_leaves'    : (300, 600),
            'max_depth'     : (2, 25),
            'min_child_weight' : (30, 100),
            'colsample_bytree' : (0, 0.99),
            'feature_fraction' : (0.0001, 0.99),
            'bagging_fraction' : (0.0001, 0.99),
            'lambda_l1'        : (0, 0.99),
            'lambda_l2'        : (0, 0.99)
        }

In [23]:
lgbmBO = BayesianOptimization(f = lgbm_cv, pbounds = pbounds, verbose = 2, random_state = 99)
lgbmBO.maximize()

|   iter    |  target   | baggin... | colsam... | featur... | lambda_l1 | lambda_l2 |    lr     | max_depth | min_ch... | num_le... |
-------------------------------------------------------------------------------------------------------------------------------------
[LightGBM] [Warning] lambda_l1 is set=0.03113192375003516, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.03113192375003516
[LightGBM] [Warning] feature_fraction is set=0.8172576727781338, colsample_bytree=0.4831976152481779 will be ignored. Current value: feature_fraction=0.8172576727781338
[LightGBM] [Warning] lambda_l2 is set=0.7999694637311993, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.7999694637311993
[LightGBM] [Warning] bagging_fraction is set=0.6655885451886208, subsample=1.0 will be ignored. Current value: bagging_fraction=0.6655885451886208
[LightGBM] [Warning] lambda_l1 is set=0.03113192375003516, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.03113192375003516
[LightGBM] [